In [1]:
# type: ignore
try:
  import sys
  sys.path.append('../')
  from modules.validate_prop import *
except ImportError:
  url = 'https://raw.githubusercontent.com/jhjeong314/Proofmood/main/modules'
  import httpimport
  with httpimport.remote_repo(url):
    from validate_prop import *

from pprint import pprint

In [2]:
# Construct an Ann object from a string.
s = " imp  intro   4-8 "
try:
  ann = Ann(s)
  print(f"input_str = '{ann.input_str}'")
  print(ann.build_str())
  print("\noutput from __str__():")
  print(" ", ann)
except Exception as e:
    print(e)  

input_str = ' imp  intro   4-8 '
rule: imp intro
premise: ['4-8']

output from __str__():
  imp intro 4-8


In [3]:
# Construct a NodeLabel object from a string.
line = "A imp B .hyp"
label = NodeLabel('formula', line)
print(label.build_str())
print()
print(label)

type: formula
line: A imp B .hyp
formula: A imp B
ann: hyp 
is_hyp: True

A imp B	 .hyp 


In [4]:
# Construct a ProofNode manually. (not by using the parser)
line1 = "A imp B .hyp"
label1 = NodeLabel('formula', line1)
node1 = ProofNode(label1)
line2 = "A .hyp"
label2 = NodeLabel('formula', line2)
node2 = ProofNode(label2)
line3 = "B .imp elim 1,2"
label3 = NodeLabel('formula', line3)
node3 = ProofNode(label3)
line4 = "A .repeat 2"
label4 = NodeLabel('formula', line4)
node4 = ProofNode(label4)
label0 = NodeLabel()
node0 = ProofNode(label0, [node1, node2, node3, node4])
node0.build_index() # necessary for printing node0
print(node0)

│1. A imp B	 .hyp 
│2. A	 .hyp 
├─
│3. B	 .imp elim 1,2
│4. A	 .repeat 2



In [5]:
# Another example, similar to the one above.
line1 = "A imp B .hyp"
label1 = NodeLabel('formula', line1)
node1 = ProofNode(label1)
line2 = "B imp C .hyp"
label2 = NodeLabel('formula', line2)
node2 = ProofNode(label2)
line3 = "A .hyp"
label3 = NodeLabel('formula', line3)
node3 = ProofNode(label3)
line4 = "B .imp elim 1,3"
label4 = NodeLabel('formula', line4)
node4 = ProofNode(label4)
line5 = "C .imp elim 2,4"
label5 = NodeLabel('formula', line5)
node5 = ProofNode(label5)
line6 = "A imp C .imp intro 3-5"
label6 = NodeLabel('formula', line6)
node6 = ProofNode(label6)
label_root = NodeLabel()
label_sub = NodeLabel()
node_sub = ProofNode(label_sub, [node3, node4, node5])
node_root = ProofNode(label_root, [node1, node2, node_sub, node6])
node_root.build_index()
print(node_root)

│1. A imp B	 .hyp 
│2. B imp C	 .hyp 
├─
││3. A	 .hyp 
│├─
││4. B	 .imp elim 1,3
││5. C	 .imp elim 2,4
│6. A imp C	 .imp intro 3-5



In [6]:
# This is a test for the preprocessing function get_str_li().
# The output of get_str_li() is a list of strings 
# to be fed to the parser.

# You may or may not prefix each line with line number.
# The parser can assign the correct line number 
# to each node anyway.
# You may also use | instead  of TAB to indent subproofs.
# The following 4 input strings are all equivalent---they 
# are parsed into the same proof tree.

prf_str1 = '''
1. A imp B .hyp
2. B imp C .hyp
proves
  3. A .hyp
  proves
  4. B     .imp elim 1,3
  5. C     .imp elim 2,4
6. A imp C .imp intro 3-5
'''
# or,
prf_str2 = '''
│1. A imp B	 .hyp
│2. B imp C	 .hyp
├─
││3. A	 .hyp
│├─
││4. B	 .imp elim 1,3
││5. C	 .imp elim 2,4
│6. A imp C	 .imp intro 3-5
'''
# or, above two without numbering
prf_str3 = '''
A imp B .hyp
B imp C .hyp
proves
  A .hyp
  proves
  B     .imp elim 1,3
  C     .imp elim 2,4
A imp C .imp intro 3-5
'''
# or,
prf_str4 = '''
│A imp B	 .hyp
│B imp C	 .hyp
├─
││A	 .hyp
│├─
││B	 .imp elim 1,3
││C	 .imp elim 2,4
│A imp C	 .imp intro 3-5
'''
# Convert either string to the following.
'''
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5
'''

for i in range(1, 5):
  prf_str_name = f"prf_str{i}"
  prf_str = eval(prf_str_name)
  print(f"[{prf_str_name}]:")
  lines = get_str_li(prf_str, tabsize=2)
  print_lines(lines)    
  print()

[prf_str1]:
A imp B .hyp
B imp C .hyp
{{
  A .hyp
  B     .imp elim 1,3
  C     .imp elim 2,4
}}
A imp C .imp intro 3-5

[prf_str2]:
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5

[prf_str3]:
A imp B .hyp
B imp C .hyp
{{
  A .hyp
  B     .imp elim 1,3
  C     .imp elim 2,4
}}
A imp C .imp intro 3-5

[prf_str4]:
A imp B	 .hyp
B imp C	 .hyp
{{
  A	 .hyp
  B	 .imp elim 1,3
  C	 .imp elim 2,4
}}
A imp C	 .imp intro 3-5



In [7]:
# Another example.

# This is a Fitch proof for 
#         A -> B |- (A or C) -> (B or C)
#
prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. A .hyp   
    proves
    4. B .imp elim 1,3
    5. B or C .or intro 4
    6. C .hyp
    proves
    7. B or C .or intro 6
  8. B or C .or elim 2,3-5,6-7
9. A or C imp B or C .imp intro 2-8'''
lines = get_str_li(prf_str, tabsize=2)
print_lines(lines)


A imp B .hyp
{{
  A or C .hyp
  {{
    A .hyp
    B .imp elim 1,3
    B or C .or intro 4
  }}
  {{
    C .hyp
    B or C .or intro 6
  }}
  B or C .or elim 2,3-5,6-7
}}
A or C imp B or C .imp intro 2-8


In [8]:
# A buggy example.

prf_str = '''
1. A imp B	 .hyp
2.
proves
  3. # subproof may start with a comment
  4. A or C	 .hyp 
  proves
  5. A	 .hyp 
  proves
  6. B	 .imp elim 1,5
  7. B or C	 .and intro 6
  8. C	 .hyp 
  proves
  9. B or C	 .or intro 8
  10. # end of subproof
  11. B or C	 .or elim 3,5-7,8-10
12. A or C imp B or C	 .imp intro 3-11
'''
try:
  lines = get_str_li(prf_str, tabsize=2)
  print_lines(lines) 
except Exception as e:
  print(e)

get_str_li(): same level and right_after_proves, but is_hyp
	Perhaps you should indent the following line:
	5. A	 .hyp


In [9]:
# This is the first test for the parser.
# Validation is done too within the parser.
# Note that we omitted the line numbers and it still works.
prf_str = '''
A .hyp
B  .hyp
proves
A and B .and intro 1,2
'''
proof = parse_fitch(prf_str)
print(proof)

│1. A	 .hyp 
│2. B	 .hyp 
├─
│3. A and B	 .and intro 1,2



In [10]:
# Show validation result.
prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. A .hyp
    proves
    4. B      .imp  elim   1,3
    5. B or C .or intro 4
    6. C .hyp
    proves
    7. B or C .or intro 6
  8. B or C .or elim 2,3-5,6-7
9. A or C imp B or C .imp intro 2-8
'''
proof = parse_fitch(prf_str)
proof.show_fitch_text(False) # False means no validation
proof.show_fitch_text(True) # True means validation

│1. A imp B	 .hyp 
├─
││2. A or C	 .hyp 
│├─
│││3. A	 .hyp 
││├─
│││4. B	 .imp elim 1,3
│││5. B or C	 .or intro 4
│││6. C	 .hyp 
││├─
│││7. B or C	 .or intro 6
││8. B or C	 .or elim 2,3-5,6-7
│9. A or C imp B or C	 .imp intro 2-8

│1. A imp B	 .hyp
├─
││2. A or C	 .hyp
│├─
│││3. A	 .hyp
││├─
│││4. B	✓ imp elim 1,3
│││5. B or C	✓ or intro 4
│││6. C	 .hyp
││├─
│││7. B or C	✓ or intro 6
││8. B or C	✓ or elim 2,3-5,6-7
│9. A or C imp B or C	✓ imp intro 2-8


In [11]:
# LaTeX output
print(proof.build_fitch_latex(1))

% \usepackage{proofmood}
\begin{fitchproof}
& \pmvert \pnumb{1} \pform{A \rightarrow B} & \chknull & \infrul{hyp} & \pmnl
& \pmproves & & & \\
& \pmvert \pmvert \pnumb{2} \pform{A \vee C} & \chknull & \infrul{hyp} & \pmnl
& \pmvert \pmproves & & & \\
& \pmvert \pmvert \pmvert \pnumb{3} \pform{A} & \chknull & \infrul{hyp} & \pmnl
& \pmvert \pmvert \pmproves & & & \\
& \pmvert \pmvert \pmvert \pnumb{4} \pform{B} & \chkch & \infrule{\rightarrow}{elim}& \pmprem{1,3} \pmnl
& \pmvert \pmvert \pmvert \pnumb{5} \pform{B \vee C} & \chkch & \infrule{\vee}{intro}& \pmprem{4} \pmnl
& \pmvert \pmvert & & & \\
& \pmvert \pmvert \pmvert \pnumb{6} \pform{C} & \chknull & \infrul{hyp} & \pmnl
& \pmvert \pmvert \pmproves & & & \\
& \pmvert \pmvert \pmvert \pnumb{7} \pform{B \vee C} & \chkch & \infrule{\vee}{intro}& \pmprem{6} \pmnl
& \pmvert \pmvert & & & \\
& \pmvert \pmvert \pnumb{8} \pform{B \vee C} & \chkch & \infrule{\vee}{elim}& \pmprem{2,3-5,6-7} \pmnl
& \pmvert & & & \\
& \pmvert \pnumb{9} \pform

In [12]:
# Show validation result.
# Added some comments and blank lines to the above proof.
# Gave wrong annotation to some lines intentionally.
prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. # comment in hypothesis
    4. A .hyp
    proves
    5. B      .imp  elim   1,4
    6. # below is a blank line
    7. 
    8. B or C .and intro 5
    9. C .hyp
    proves
    10. B or C .or intro 1
  11. B or C .or elim 2,3-8,9-10
12. A or C imp B or C .imp intro 2-11
'''
try:
  proof = parse_fitch(prf_str)
  proof.show_fitch_text()
  proof.show_fitch_text(True)
  print()
except Exception as e:
  print(e)

│1. A imp B	 .hyp
├─
││2. A or C	 .hyp
│├─
│││3. # comment in hypothesis
│││4. A	 .hyp
││├─
│││5. B	✓ imp elim 1,4
│││6. # below is a blank line
│││7. 
│││8. B or C	x and intro 5
│││9. C	 .hyp
││├─
│││10. B or C	x or intro 1
││11. B or C	✓ or elim 2,3-8,9-10
│12. A or C imp B or C	✓ imp intro 2-11
│1. A imp B	 .hyp
├─
││2. A or C	 .hyp
│├─
│││3. # comment in hypothesis
│││4. A	 .hyp
││├─
│││5. B	✓ imp elim 1,4
│││6. # below is a blank line
│││7. 
│││8. B or C	x and intro 5
│││9. C	 .hyp
││├─
│││10. B or C	x or intro 1
││11. B or C	✓ or elim 2,3-8,9-10
│12. A or C imp B or C	✓ imp intro 2-11



In [13]:
# proof.index_dict is a dictionary that maps line numbers 
# to indices.
index_dict = proof.build_index_dict()
pprint(index_dict, sort_dicts=False)

{'1-12': [0],
 '1': [0, 0],
 '2-11': [0, 1],
 '2': [0, 1, 0],
 '3-8': [0, 1, 1],
 '3': [0, 1, 1, 0],
 '4': [0, 1, 1, 1],
 '5': [0, 1, 1, 2],
 '6': [0, 1, 1, 3],
 '7': [0, 1, 1, 4],
 '8': [0, 1, 1, 5],
 '9-10': [0, 1, 2],
 '9': [0, 1, 2, 0],
 '10': [0, 1, 2, 1],
 '11': [0, 1, 3],
 '12': [0, 2]}


In [14]:
# The order relation between nodes.
# proof := parse_fitch(prf_str)
# index_dict = proof.build_index_dict()
for i in index_dict:
  for j in index_dict:
    label_j = proof.get_p_node(j).label
    if label_j.type == 'formula' and not label_j.is_hyp: # conclusion
      if proof.is_earlier(i, j):
        print(f"{i} <= {j}") 

1 <= 5
1 <= 8
1 <= 10
1 <= 11
1 <= 12
2-11 <= 12
2 <= 5
2 <= 8
2 <= 10
2 <= 11
3-8 <= 10
3-8 <= 11
3 <= 5
3 <= 8
4 <= 5
4 <= 8
5 <= 8
6 <= 8
7 <= 8
9-10 <= 11
9 <= 10


In [15]:
# FormulaProp.verified_by()

fmla = FormulaProp('B1(x)')
node1 = parse_ast('(A iff not C_1) imp B1(x)')
node2 = parse_ast('A iff not C_1')
print(fmla.verified_by(RuleInfer.IMP_ELIM, [node1, node2]))
print(fmla.verified_by(RuleInfer.IMP_ELIM, [node2, node1]))
print(fmla.verified_by(RuleInfer.AND_INTRO, [node1, node2]))

True
True
False


In [16]:
# FormulaProp.verified()

# A imp B proves A or C imp B or C
# annotation for line 5 is intentionally wrong
prf_str = '''
1. A imp B .hyp
proves
  2. A or C .hyp
  proves
    3. A .hyp
    proves
    4. B .imp elim 1,3
    5. B or C .and intro 4
    6. C .hyp
    proves
    7. B or C .or intro 6
  8. B or C .or elim 2,3-5,6-7
9. A or C imp B or C .imp intro 2-8
'''
proof = parse_fitch(prf_str)
print(proof.verified("4"))
print(proof.verified("5"))

True
False


In [17]:
proof.verified_all()
#^ This gives false because there is a line with wrong 
# annotation: namely, line 5.

False